In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\D Drive\Courses\Non Umich Courses\CMU\Optimal_Control_16_745\Lectures and Codes\Lecture 13\Project.toml`


In [2]:
using LinearAlgebra
using ForwardDiff

In [3]:
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end

hat (generic function with 1 method)

In [4]:
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end

L (generic function with 1 method)

In [5]:
function R(q)
    s = q[1]
    v = q[2:4]
    R = [s    -v';
         v  s*I-hat(v)]
    return R
end

R (generic function with 1 method)

In [6]:
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I];

In [7]:
function G(q)
    G = L(q)*H
end

function Q(q)
    return H'*(R(q)'*L(q))*H
end

Q (generic function with 1 method)

In [8]:
J = Diagonal([1; 2; 3])
h = 0.1

0.1

In [9]:
#initial conditions
Q0 = Array(I(3))
q0 = [1; 0; 0; 0]
ω0 = randn(3)
x0 = [vec(Q0); ω0]
x0q = [q0; ω0]

7-element Vector{Float64}:
  1.0
  0.0
  0.0
  0.0
 -2.2407928047750922
 -0.23673441340621923
  0.5954444772156684

In [10]:
#dynamics
function dynamics(x)
    Q = reshape(x[1:9],3,3)
    ω = x[10:12]
    
    Q̇ = Q*hat(ω)
    ω̇ = -J\(hat(ω)*J*ω)

    ẋ = [vec(Q̇); ω̇]
end

dynamics (generic function with 1 method)

In [11]:
function rkstep(x)
    f1 = dynamics(x)
    f2 = dynamics(x + 0.5*h*f1)
    f3 = dynamics(x + 0.5*h*f2)
    f4 = dynamics(x + h*f3)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    return xn
end

rkstep (generic function with 1 method)

In [12]:
xk = x0
for k = 1:10000
    xk = rkstep(xk)
end

In [13]:
Qk = reshape(xk[1:9],3,3)

3×3 Matrix{Float64}:
  0.100288     -0.942715   -0.304359
  0.000405349  -0.306557    0.94113
 -0.989348     -0.0853291  -0.0295668

In [14]:
Qk'*Qk

3×3 Matrix{Float64}:
  0.988868   -0.0102469    -0.0008901
 -0.0102469   0.98997       0.000936703
 -0.0008901   0.000936703   0.979235

In [15]:
#quaternion dynamics
function qdynamics(x)
    q = x[1:4]
    ω = x[5:7]
    
    q̇ = 0.5*L(q)*H*ω
    ω̇ = -J\(hat(ω)*J*ω)

    ẋ = [q̇; ω̇]
end

qdynamics (generic function with 1 method)

In [16]:
function qrkstep(x)
    f1 = qdynamics(x)
    f2 = qdynamics(x + 0.5*h*f1)
    f3 = qdynamics(x + 0.5*h*f2)
    f4 = qdynamics(x + h*f3)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[1:4] .= xn[1:4]./norm(xn[1:4])
    return xn
end

qrkstep (generic function with 1 method)

In [17]:
xkq = x0q
for k = 1:10000
    xkq = qrkstep(xkq)
end

In [18]:
qk = xkq[1:4]

4-element Vector{Float64}:
 -0.41752204204890503
  0.6145699356285862
 -0.39498777651418093
 -0.5403367422535427

In [19]:
norm(qk)

1.0